In [1]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from torch.utils.data import random_split, DataLoader
from tqdm import tqdm

Я использовал Google COlab, чтобы тренировать модель

In [2]:
if torch.cuda.is_available():
  device = torch.device("cuda")
  print("CUDA is available. Using GPU.")
else:
  device = torch.device("cpu")
  print("CUDA is not available. Using CPU.")

CUDA is not available. Using CPU.


In [3]:
PATH_TO_IMAGES = '../../images'

Изображения перед обучением модели слегка поменял (чтобы было больше разнообразия)

In [ ]:
preprocess = transforms.Compose(
    [
      transforms.Resize(512),
      transforms.RandomRotation(20),
      transforms.RandomHorizontalFlip(),
      transforms.RandomVerticalFlip(),
      transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),

      transforms.Resize(256),
      transforms.CenterCrop(224),
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

dataset = torchvision.datasets.ImageFolder(
    root=PATH_TO_IMAGES,
    transform=preprocess
)

Датасет поделил на учебный и тестовый

In [5]:
train_dataset, test_dataset = random_split(dataset, [0.8, 0.2])

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)

Загрузка готовой предобученной модели resnet50

In [6]:
resnet50_model = torchvision.models.resnet50(
    weights=torchvision.models.ResNet50_Weights.IMAGENET1K_V1
)

Можно посмотреть, из чего она состоит

In [7]:
resnet50_model.eval()
print(resnet50_model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

Последнй слой (fully connected) заменим на nn.Identity(), который ничего не делает, а передает выход на вход

In [8]:
resnet50_model.fc = nn.Identity()
resnet50_model.eval()

print(resnet50_model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

Вместо него сделаем собстенныеслои: на входе 2048 нейноров, скрытый слой, на выходе 1 нейрон-классификатор

In [9]:
fc_model = nn.Sequential(
    nn.Linear(in_features=2048, out_features=1024),
    nn.ReLU(),
    nn.Linear(in_features=1024, out_features=512),
    nn.ReLU(),
    nn.Linear(in_features=512, out_features=1)
)

И теперь последовательно соединим слои: resnet50 с замененный выходным слоем и свой fc_model

In [10]:
model = nn.Sequential(
    resnet50_model,
    fc_model
)

Переведем resnet в режим eval - мы ее не учим; опделелим оптимизатор и шаг обучения

In [12]:
resnet50_model.eval()
for param in resnet50_model.parameters():
  param.requires_grad = False

optimizer = torch.optim.Adam(fc_model.parameters(), lr=0.001)

.... и функцию потерь

In [13]:
loss_fn = nn.BCEWithLogitsLoss()

Загоним все в определенный ранее device

In [14]:
resnet50_model = resnet50_model.to(device)
fc_model = fc_model.to(device)
model = model.to(device)

Собственно, обучение. Эпох я здесь поставил 2, но при обучении использовал 15 эпох <br>
Тут интересно - после обучения сразу делаем evaluation - можно видеть, как идет процесс обучения<br>
Плюс - промежуточные модели сохраняются, чтобы можно было выбрать лучшую

In [35]:
NUMBER_OF_EPOCH = 2
for epoch in range(NUMBER_OF_EPOCH):
  print(f'----------Epoch: {epoch} ({NUMBER_OF_EPOCH}) ---------')
  model.train()
  resnet50_model.eval()

  loss_sum = 0
  for X, y in tqdm(train_dataloader):
    X = X.to(device)
    y = y.to(device).type(torch.float).reshape(-1, 1)  # это нужно, чтоб скормить y resnet'у

    out = model(X)
    optimizer.zero_grad()
    loss = loss_fn(out, y)
    loss_sum += loss.item()

    loss.backward()
    optimizer.step()

  print(f'Training loss: {loss_sum/len(train_dataloader)}')

  torch.save(fc_model.state_dict(), f'model_weights_{epoch}.pth')
  model.eval()
  val_loss_sum = 0

  with torch.no_grad():
    for X, y in test_dataloader:
      X = X.to(device)
      y = y.to(device).type(torch.float).reshape(-1, 1)

      out = model(X)
      loss = loss_fn(out, y)
      val_loss_sum += loss.item()

      prediction = torch.sigmoid(out) > 0.5
      accuracy = (prediction == y).sum()
      print(accuracy/len(y))
      break

    print(f'Test Loss: {val_loss_sum/len(test_dataloader)}')

  print(loss_sum/len(train_dataloader))

----------Epoch: 0 (2) ---------


100%|██████████| 47/47 [12:29<00:00, 15.94s/it]


Training loss: 0.239504910529928
tensor(0.7812)
Test Loss: 0.027874372899532318
0.239504910529928
----------Epoch: 1 (2) ---------


100%|██████████| 47/47 [11:43<00:00, 14.97s/it]


Training loss: 0.24402527091033915
tensor(0.9062)
Test Loss: 0.012591073910395304
0.24402527091033915


После обучения можно выбрать лучшую модель и загрузить ее веса в модель.<br>
Здесь для примера взята первая из сохраненных: явно не лучшая

In [ ]:
weights = torch.load(
    'model_weights_14.pth',
    weights_only=True,
    map_location=torch.device('cpu')
)

torch.save(weights, 'model_weights_14_cpu.pth')

FileNotFoundError: [Errno 2] No such file or directory: 'model_weights_14.pth'

Теперь можно брать любые фоты, делать их квадратными, нормализовать

In [37]:
preprocess = transforms.Compose(
    [
      transforms.Resize((256, 256)),
      transforms.CenterCrop(224),
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

Опять пересобрать модель (этот код уже в Джанге можно использовать)

In [38]:
resnet50_model = torchvision.models.resnet50(
    weights=torchvision.models.ResNet50_Weights.IMAGENET1K_V1
)
resnet50_model.fc = nn.Identity()
resnet50_model = resnet50_model.to(device)

fc_model = nn.Sequential(
    nn.Linear(in_features=2048, out_features=1024),
    nn.ReLU(),
    nn.Linear(in_features=1024, out_features=512),
    nn.ReLU(),
    nn.Linear(in_features=512, out_features=1)
)

fc_statedict = torch.load(
    'model_weights_14_cpu.pth',
    weights_only=True
)

fc_model.load_state_dict(fc_statedict)
fc_model = fc_model.to(device)

In [39]:
model = nn.Sequential(
    resnet50_model,
    fc_model
).to(device)
model.eval()

Sequential(
  (0): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0)

Теперь берем любое изображение, обрабатываем его и скрармливаем модели. <br>
Результат пропускаем через функцию активации (sigmoid)

In [40]:
from PIL import Image

In [41]:
path = '../../images/good/good (186).jpg'
image = Image.open(path)
image_tensor = preprocess(image)
image_tensor = image_tensor.unsqueeze(dim=0)
image_tensor = image_tensor.to(device)

with torch.no_grad():
  out = model(image_tensor)
  y_pred = torch.sigmoid(out)
  print(y_pred)
  print(y_pred > 0.5)

tensor([[0.9708]])
tensor([[True]])


На выходе имеем класс и вероятность. Ее можно выводить на страницу.

Здесь повтор кода

In [30]:
image_tensor = preprocess(image)
image_tensor = image_tensor.unsqueeze(dim=0)
image_tensor.shape

torch.Size([1, 3, 224, 224])

In [32]:
image_tensor = image_tensor.to(device)

In [33]:
with torch.no_grad():
  out = model(image_tensor)
  print(out)
  y_pred = torch.sigmoid(out)
  print(y_pred)
  print(y_pred > 0.5)

tensor([[5.2711]])
tensor([[0.9949]])
tensor([[True]])
